In [2]:
import numpy as np
import pandas as pd
import string
#!pip install gensim

In [3]:
df = pd.read_excel("Dirty data from RFP COE.xlsx")
df.head()

,Non-Functional Requirement,Description
0,Is system based on a multi-tiers architecture?...,Depending on the current state and future sele...
1,How is the system architected? (i.e. is it bas...,For more information on the current architectu...
2,Is it possible to add further modules in the f...,Extensibility covers a broad spectrum of topic...
3,Please provide a detailed interface descriptio...,SAP API Management provides enterprises a comp...
4,Is it possible to make the system data availab...,SAP supports all standard integration capabili...


In [4]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716 entries, 0 to 716
Data columns (total 2 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Non-Functional Requirement  716 non-null    object
 1   Description                 716 non-null    object
dtypes: object(2)
memory usage: 16.8+ KB


'Is system based on a multi-tiers architecture? If yes, how many tiers are there and what does each tier do?'

In [5]:
from re import sub
from gensim.utils import simple_preprocess


stopwords = ['the', 'and', 'are', 'a']

def preprocess(doc):
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]


In [6]:
corpus = [preprocess(q) for q in df['Non-Functional Requirement']]

['is',
 'system',
 'based',
 'on',
 'multi',
 'tiers',
 'architecture',
 'if',
 'yes',
 'how',
 'many',
 'tiers',
 'there',
 'what',
 'does',
 'each',
 'tier',
 'do']

In [7]:
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

In [8]:
#Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)
query = input("Enter the Query")
query = preprocess(query)
# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

Enter the QueryIs system based on a multi-tiers architecture? If yes, how many tiers are there and what does each tier do?


In [9]:

query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]


sorted_indexes = np.argsort(doc_similarity_scores)[-4:][::-1]
for idx in sorted_indexes[1:]:
    print(df.iloc[idx,0],"with the accuracy ",doc_similarity_scores[idx])
    print('-'*400)

Are all OS for device supported by the application? What are the OS supported for devices to access the application? with the accuracy  0.7737855
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
What tools are provided for archiving data? with the accuracy  0.6897444
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

/home/tauseefnawaz/anaconda3/lib/python3.7/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/home/tauseefnawaz/anaconda3/lib/python3.7/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
